In [1]:
from scipy.signal import butter, lfilter
import pandas as pd
import numpy as np
import csv
import glob
import matplotlib.pyplot as plt
import datetime
from zipfile import ZipFile

In [2]:
def get_utc_date_time(ts):
    return pd.to_datetime(ts, unit='s', infer_datetime_format = True, utc = True)
#.strftime('%H:%M:%S:%f')

def add_fs(sample_rate, date):
    return date + datetime.timedelta(milliseconds=1.0/(sample_rate) * 1000.0)

In [3]:
def time_range(df_length, start_timestamp, sample_rate):
    # Generate date time range based on sample rate and starting timestamp
    time_range = []
    t_0 = get_utc_date_time(float(start_timestamp))
    time_range.append(t_0)
    
    next_date = t_0
    
    for count in range(df_length-1):
        next_date = add_fs(sample_rate, next_date)
        time_range.append(next_date)
    return time_range

In [4]:
def format_time(t):
    s = t.strftime('%Y-%m-%d %H:%M:%S.%f')
    return s[:-3]

In [ ]:
def process_tags(list_of_zips, participant_id):
    tags_all = []

    for zip_file in list_of_zips:
        zf = ZipFile(zip_file)
        tags_df_raw = pd.read_csv(zf.open('tags.csv'),header=None,names=["TAGS"])   

        tags_df = tags_df_raw.copy()

        tags_df['TAGS'] = get_utc_date_time(tags_df['TAGS'])
        tags_df['TAGS'] = tags_df.apply(lambda x: format_time(x['TAGS']) + '+00:00', axis=1)
        tags_df['ParticipantId'] = participant_id

        tags_all.append(tags_df)
    
    tags_df_all = []
    tags_df_all = pd.concat(tags_all)
    return tags_df_all

In [5]:
base_path = r"C:\Users\shaye\Documents\EAT Lab\Sensor_Data\Empatica data"
participant_data = glob.glob(base_path + "/ID PR*")

'C:\\Users\\shaye\\Documents\\EAT Lab\\Sensor_Data\\Empatica data'

In [7]:
for path in participant_data: 
    path_parts = path.split("\\")
    participant_id = path_parts[-1:][0].split()[-1:][0]
    list_of_zips = glob.glob(path + "/Empatica data/A*.zip")
    tags_test = process_tags(list_of_zips, participant_id)
    print(tags_test)